In [1]:
# for more info see README.md

# necessary functions
import csv

def extract_color_data(roi_result_models, data_series_type_target):
    """Extract color data based on RoiModel and DataResultsModels for a given DataSeriesType."""
    color_data = {"X": [], "Red": [], "Green": [], "Blue": []}
    x_data_added = False  # Flag to check if x_data has been added
    
    for item in roi_result_models:
        data_results_models = item.get("DataResultsModels", [])
        color = item.get("RoiModel", {}).get("Color", "")

        for data_item in data_results_models:
            data_series_type = data_item.get("DataSeriesType", 0)
            if data_series_type == data_series_type_target:
                x_data = data_item.get("X", [])
                y_data = data_item.get("Y", [])
                if not x_data_added:
                    color_data["X"].extend(x_data)
                    x_data_added = True 
                color_data[color].extend(y_data)
                
    return color_data


def write_to_csv(file_path, color_data):
    """Write color data to a CSV file."""
    with open(file_path, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(["X", "H", "R", "L"])
        
        max_length = max(len(color_data["X"]), len(color_data["Red"]), len(color_data["Green"]), len(color_data["Blue"]))
        for i in range(max_length):
            row = [
                color_data["X"][i] if i < len(color_data["X"]) else None,
                color_data["Red"][i] if i < len(color_data["Red"]) else None,
                color_data["Green"][i] if i < len(color_data["Green"]) else None,
                color_data["Blue"][i] if i < len(color_data["Blue"]) else None
            ]
            csvwriter.writerow(row)
def extract_desired_folder_name(path):
    # Convert backslashes to forward slashes for consistent path handling
    path = path.replace("\\", "/")
    # Split the path into its components
    parts = path.split("/")
    # Check if there are at least two folders above the filename
    if len(parts) >= 3:
        return parts[-3]  # Return the desired folder name
    return None  # Return None if extraction is not possible


In [6]:
import os
import json


# Define paths and get the list of folders in AllM with a prefix of v_ Used by treman app
pathToAllM = os.path.join( os.environ['USERPROFILE'], 'AppData', 'Local', 'Packages', '26872JanTesa.TremAn3_pc8qdhpdg0sz8', 'LocalState', 'AllM')


v_folders = [folder for folder in os.listdir(pathToAllM) if folder.startswith("v_")]

# Extract the Path property for each v_ folder
folder_path_mapping = {}
for v_folder in v_folders:
    json_file_path = os.path.join(pathToAllM, v_folder, f"{v_folder}.json")
    if os.path.exists(json_file_path):
        with open(json_file_path, 'r') as json_file:
            data = json.load(json_file)
            folder_path_mapping[v_folder] = data.get("Path", "")

# Extract the desired folder names from the paths
desired_folder_names = [extract_desired_folder_name(path) for path in folder_path_mapping.values()]

# Initialize a dictionary to store the extracted data for each v_ folder
extracted_data = {}

# Define the data_path
data_path = 'data'

# Modify the CSV files and save the ROI data to JSON files

roi_info = {}  # To store the RoiModel information for JSON files

# Loop through the desired DataSeriesTypes and call the functions to handle extraction and CSV generation
data_series_types = {"ComX": 1, "ComY": 2}
for file_name, dst in data_series_types.items():
    for v_folder in v_folders:
        # Extract color data
        m_folder = [folder for folder in os.listdir(os.path.join(pathToAllM, v_folder)) if folder.startswith("m_")][0]
        vector_data_file = [file for file in os.listdir(os.path.join(pathToAllM, v_folder, m_folder)) if file.endswith("_vectorData.json")][0]
        
        with open(os.path.join(pathToAllM, v_folder, m_folder, vector_data_file), 'r') as json_file:
            data = json.load(json_file)
            roi_result_models = data.get("RoiResultModels", [])
            
            color_data = extract_color_data(roi_result_models, dst)
            # Extract ROI information for JSON file
            rois = []
            for item in roi_result_models:
                roi_model = item.get("RoiModel", {})
                color = roi_model.get("Color", "")
                column = "H" if color == "Red" else "R" if color == "Green" else "L"
                
                roi = {
                    "x": roi_model.get("X", 0),
                    "y": roi_model.get("Y", 0),
                    "width": roi_model.get("Width", 0),
                    "height": roi_model.get("Height", 0),
                    "Column": column
                }
                rois.append(roi)
        
        roi_info[v_folder] = rois
        
        # Determine the save path for CSV
        video_name = os.path.basename(folder_path_mapping[v_folder])
        save_folder = os.path.join(data_path, desired_folder_names[v_folders.index(v_folder)], os.path.splitext(video_name)[0])
        if not os.path.exists(save_folder):
            os.makedirs(save_folder)
        save_path = os.path.join(save_folder, f"{file_name}.csv")
        
        # Write to CSV
        write_to_csv(save_path, color_data)



# Save the ROI data to JSON files
for v_folder, rois in roi_info.items():
    video_name = os.path.basename(folder_path_mapping[v_folder])
    save_folder = os.path.join(data_path, desired_folder_names[v_folders.index(v_folder)], os.path.splitext(video_name)[0])
    
    # Extract FrameRate from the other JSON
    m_folder = [folder for folder in os.listdir(os.path.join(pathToAllM, v_folder)) if folder.startswith("m_")][0]
    json_files = [file for file in os.listdir(os.path.join(pathToAllM, v_folder, m_folder)) if file.endswith(".json") and not file.endswith("_vectorData.json")]
    
    if json_files:
        measurement_data_file = json_files[0]
        
        # Read the JSON content
        with open(os.path.join(pathToAllM, v_folder, m_folder, measurement_data_file), 'r') as json_file:
            data = json.load(json_file)
            frame_rate = data.get("FrameRate", 0)
    
    # Adjust the rois data to include FrameRate
    info_data = {
        "FrameRate": frame_rate,
        "ROIs": rois
    }
    
    save_path = os.path.join(save_folder, "info.json")
    
    # Save the ROI data along with FrameRate to the JSON file
    with open(save_path, 'w') as json_file:
        json.dump(info_data, json_file, indent=4)
